In [2]:
import pandas as pd
import re
import tqdm as notebook_tqdm
from transformers import AutoModel, AutoTokenizer
from razdel import tokenize
from string import punctuation
import torch


emoj = re.compile('['
                  u'\U0001F600-\U0001F64F'
                  u'\U0001F300-\U0001F5FF'
                  u'\U0001F680-\U0001F6FF'
                  u'\U0001F1E0-\U0001F1FF'
                  u'\U00002500-\U00002BEF'
                  u'\U00002702-\U000027B0'
                  u'\U000024C2-\U0001F251'
                  u'\U0001f926-\U0001f937'
                  u'\U00010000-\U0010ffff'
                  u'\u2640-\u2642'
                  u'\u2600-\u2B55'
                  u'\u200d'
                  u'\u23cf'
                  u'\u23e9'
                  u'\u231a'
                  u'\ufe0f'
                  u'\u3030'
                  ']', re.UNICODE)
punctuation += '—'
tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
vectorizer = AutoModel.from_pretrained("ai-forever/sbert_large_nlu_ru")
with open('../DATA/less_semantic_words.txt', 'r', encoding='utf-8') as f:
    stop_words = pd.Series(f.read().split('\n')).unique()

In [5]:
def clear_from_stopwords(text, sw):
    text = re.sub(emoj, '', text)
    tokens = list(tokenize(text))
    t = []
    for i in tokens:
        if len(i.text) > 2 and i.text not in sw:
            t.append(i.text)
    return ' '.join(t)

In [12]:
check_text = vectorizer(**tokenizer([clear_from_stopwords('Программировать на Python легко', stop_words)], padding=True, truncation=True, pad_to_multiple_of=65, max_length=65, return_tensors='pt'))[0].squeeze(0).sum(dim=0)

In [22]:
t = 0
d = dict()
for i in torch.load('../DATA/small_description_vectors.pt'):
    d[t] = torch.nn.CosineSimilarity(dim=1)((i.sum(dim=0)/i.shape[0]).unsqueeze(0), check_text.unsqueeze(0)).item()
    t+=1

In [30]:
data = pd.read_excel('../DATA/Профессии_ОКСО_ ЛЦТ_Профилум.xlsx', index_col='№').loc[:, ['Отрасль текстом', 'Название профессии']]


In [29]:
data.iloc[pd.Series(d).sort_values(ascending=0).index]

,Отрасль текстом,Название профессии
№,,
22,Информационные технологии (ИТ) и телекоммуникации,Специалист по машинному обучению
21,Информационные технологии (ИТ) и телекоммуникации,Frontend-разработчик
20,Информационные технологии (ИТ) и телекоммуникации,Системный аналитик
48,Электроника и приборостроение,Разработчик систем компьютерного зрения
24,Информационные технологии (ИТ) и телекоммуникации,Разработчик мобильных приложений
17,Информационные технологии (ИТ) и телекоммуникации,Архитектор интеллектуальных систем управления ...
38,Образование и наука,Разработчик онлайн-курсов
23,Информационные технологии (ИТ) и телекоммуникации,Разработчик баз данных
27,Машиностроение и производство транспортных сре...,Фрезеровщик
